In [1]:
#tensorflow, keras
import keras
from keras.layers import merge, Conv2D, Input, Reshape, Activation
from keras.models import Model, load_model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.core import Lambda, Flatten, Dense
from keras.utils.vis_utils import plot_model
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
from numpy import genfromtxt
from keras.preprocessing.image import img_to_array as img_to_array
from keras.preprocessing.image import load_img as load_img

#essentials
import numpy as np
import math
import os
import random
import pandas as pd
import tensorflow as tf
import argparse
from keras import callbacks
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
def ConvNet(input_shape):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    # First Block
    #CONV layer
    X = Conv2D(32, (7, 7), strides = (1,1), activation='relu', padding='same')(X_input)
    # MAXPOOL + BatchNorm
    X = MaxPooling2D((2,2), strides = 2, padding='same')(X)
    X = BatchNormalization(axis=-1)(X)
    
    # Second Block
    #CONV layer
    X = Conv2D(64, (5, 5), strides = (2,2), activation='relu', padding='same')(X)
    # MAXPOOL
    X = MaxPooling2D((2,2), strides = 2, padding='same')(X)
    X = BatchNormalization(axis=-1)(X)
    
    # Third Block
    X = Conv2D(128, (5,5), strides = (1, 1),activation='relu', padding='same')(X)
    # MAXPOOL
    X = MaxPooling2D(pool_size = 3, strides = 2, padding='same')(X)
    X = BatchNormalization(axis=-1)(X)
    
    # Top layer
    X = AveragePooling2D(pool_size=(2,2), strides=(2,2))(X)
    X = Conv2D(64, (7,7), strides = (2,2),activation='relu')(X)
    
    # L2 normalization
    X = Lambda(lambda  x: K.l2_normalize(x,axis=-1))(X)

    #Final output layer. First Unit is a sigmoid act(whether seen img is infected/not)
    # next 2 units for identifying type of infection if 1st element is 1. otherwise, don't care.
    
    infected = Conv2D(2, (1, 1), strides = (1,1),activation='softmax')(X)
    infection_type = Conv2D(3, (1, 1), strides = (1,1),activation='softmax')(X)
    
    infected= Reshape((2,))(infected)
    infection_type= Reshape((3,))(infection_type)
    # Create model instance
    
    model = Model(inputs = X_input, outputs = (infected, infection_type))
    
    return model

In [3]:
def load_image(image_path, size=256):
    # data augmentation logic such as random rotations can be added here
    return img_to_array(load_img(image_path, target_size=(size, size, 3))) / 255.

class InfectedLeavesSequence(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, X, y2, batch_size=512, dim=(256,256), n_channels=3, shuffle=True):
        #Initialization
        self.dim = dim
        self.X= X
        self.y2= y2
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Find list of IDs
        batched_image_names = [self.X[k] for k in indexes]
        batched_y2 = self.y2[indexes]
        
        # Generate data
        X = self.__data_generation(batched_image_names)
        
        return X, batched_y2

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.X))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, images):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.dim[0], self.dim[1], self.n_channels))
        
        # Generate data
        for i, ID in enumerate(images):
            # Store sample
            X[i,:, :, :] = load_image('data_images/Archive/'+ID)

        return X

In [4]:
job_dir = ''

random.seed(0)
from sklearn.model_selection import StratifiedKFold

In [5]:
def prep_data(df_path='infectedLeaves.csv', n_classes1= 2, n_classes2= 3):
    df = pd.read_csv(df_path)
    #resolve priyanka's error.
    df['infectionType']= df['infectionType'].map({0: 0, 1:1, 2:2, 4:3})
    #create categorical quants
    y2= keras.utils.to_categorical(df['infectionType'], num_classes=n_classes2+1)
    #drop final axis=-1 last dim of y2
    #final images list names
    X = df['pathName'].values
    return df, X, y2

In [6]:
df, X, y2= prep_data()

In [13]:
y2

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [25]:
#data distribution
print ('total number of uninfected leaves', sum(df['infected']==0))
print ('total number of infected leaves', sum(df['infected']==1))
print ('total number of leaves with Disease1', sum(df['infectionType']==0))
print ('total number of leaves with Disease1', sum(df['infectionType']==1))
print ('total number of leaves with Disease1', sum(df['infectionType']==2))


#

total number of uninfected leaves 63003
total number of infected leaves 56997
total number of leaves with Disease1 25738
total number of leaves with Disease1 22000
total number of leaves with Disease1 9259


In [26]:
def loss(y_true, y_pred, N = 128, beta=128, epsilon=1e-8):
    
    infection_type = y_pred
    
    # loss for either layers
    return tf.losses.softmax_cross_entropy(y_true, infection_type)

In [10]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

cvscores = []

##Setting up the path for saving logs
logs_path = job_dir + 'logs/tensorboard'

In [21]:
sum(y2[:, 2]==1)

9259

In [24]:
for index, (train_indices, val_indices) in enumerate(kfold.split(X, df['infectionType'])):
    print ('KFold split: Fold', (index+1))
    print ('Training dataset size', len(train_indices))
    print ('Val dataset size', len(val_indices))
    y2_train, y2_val = y2[train_indices], y2[val_indices]
    print('')
    print ('Train: Un-infected samples', np.sum(y2_train[:, 3]==1))
    print ('Train: Disease 1 samples', np.sum(y2_train[:, 0]==1))
    print ('Train: Disease 2 samples', np.sum(y2_train[:, 1]==1))
    print ('Train: Disease 3 samples', np.sum(y2_train[:, 2]==1))
    print('')
    print ('Train: Un-infected samples', np.sum(y2_val[:, 3]==1))
    print ('Train: Disease 1 samples', np.sum(y2_val[:, 0]==1))
    print ('Train: Disease 2 samples', np.sum(y2_val[:, 1]==1))
    print ('Train: Disease 3 samples', np.sum(y2_val[:, 2]==1))
    print('')
    print('')

KFold split: Fold 1
Training dataset size 95999
Val dataset size 24001

Train: Un-infected samples 50402
Train: Disease 1 samples 20590
Train: Disease 2 samples 17600
Train: Disease 3 samples 7407

Train: Un-infected samples 12601
Train: Disease 1 samples 5148
Train: Disease 2 samples 4400
Train: Disease 3 samples 1852


KFold split: Fold 2
Training dataset size 95999
Val dataset size 24001

Train: Un-infected samples 50402
Train: Disease 1 samples 20590
Train: Disease 2 samples 17600
Train: Disease 3 samples 7407

Train: Un-infected samples 12601
Train: Disease 1 samples 5148
Train: Disease 2 samples 4400
Train: Disease 3 samples 1852


KFold split: Fold 3
Training dataset size 95999
Val dataset size 24001

Train: Un-infected samples 50402
Train: Disease 1 samples 20590
Train: Disease 2 samples 17600
Train: Disease 3 samples 7407

Train: Un-infected samples 12601
Train: Disease 1 samples 5148
Train: Disease 2 samples 4400
Train: Disease 3 samples 1852


KFold split: Fold 4
Training da

In [28]:
## Initializing the model
model = ConvNet((256, 256, 3))

#load weights
model.load_weights('checkpointSave.hdf5')

## Compling the model
model.compile(optimizer = "adam" , loss = loss, metrics = ["accuracy"]);

penultimate= model.layers[-5].output
typeOfInfection = Conv2D(4, (1, 1), strides = (1,1),activation='softmax')(penultimate)
reshapeOut= Reshape((4,))(typeOfInfection)

new_model = Model(inputs= model.input, outputs= reshapeOut)
new_model.compile(optimizer="adam", loss= 'categorical_crossentropy', metrics=['accuracy'])

print(new_model.summary())    

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 256, 256, 32) 4736        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 128, 128, 32) 0           conv2d_7[0][0]                   
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 128, 128, 32) 128         max_pooling2d_4[0][0]            
__________________________________________________________________________________________________
conv2d_8 (

In [44]:
for layer in new_model.layers:
    print(layer)

In [47]:
##Using the GPU
with tf.device('/device:GPU:0'):

    tf.reset_default_graph()
    
    
    ## Initializing the model
    model = ConvNet((256, 256, 3))

    #load weights
    model.load_weights('checkpointSave.hdf5')

    ## Compling the model
    model.compile(optimizer = "adam" , loss = loss, metrics = ["accuracy"]);

    penultimate= model.layers[-5].output
    typeOfInfection = Conv2D(4, (1, 1), strides = (1,1),activation='softmax')(penultimate)
    reshapeOut= Reshape((4,))(typeOfInfection)

    new_model = Model(inputs= model.input, outputs= reshapeOut)
    new_model.compile(optimizer="adam", loss= 'categorical_crossentropy', metrics=['accuracy'])

    print(new_model.summary())    

    filepath="softmaxChangeCheckpoint.hdf5"
    checkpoint= ModelCheckpoint(filepath, verbose=1, save_best_only=False, save_weights_only=True, mode='auto', period=1)
    callbacks_list = [checkpoint]    

    # Loop through the indices the split() method returns
    for index, (train_indices, val_indices) in enumerate(kfold.split(X, df['infectionType'])):

        print("Training on fold " + str(index+1) + "/5...")
        #print ("Evaluating on fold" + str(index+1) +"/5")
        m_train= len(train_indices)
        m_val= len(val_indices)
        
        X_train, X_val = X[train_indices], X[val_indices]
        y2_train, y2_val = y2[train_indices], y2[val_indices]
        
        train_seq = InfectedLeavesSequence(X_train, y2_train)
        val_seq = InfectedLeavesSequence(X_val, y2_val)

       # print(model.evaluate_generator(val_seq)
        X_train_1 = X_train[:100]
        y_train_1 = y2_train[:100]
        
        X = np.empty((100, 256,256,3))
        
        # Generate data
        for i, ID in enumerate(X_train_1):
            # Store sample
            X[i,:, :, :] = load_image('data_images/Archive/'+ID)

            
        new_model.fit(X, y_train_1)
        
        #save model
        print('exception thrown')
        new_model.save('SoftmaxModel_fold'+str(index+1)+'.h5')            

        #save model
        new_model.save('SoftMaxModel_fold'+str(index+1)+'.h5')

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder_3:0", shape=(32,), dtype=float32) is not an element of this graph.